<div style="text-align: center;">
    <h1><strong>UNIVERSIDAD POLITÉCNICA DE CARTAGENA</strong></h1>
    <h2><strong>GRADO EN CIENCIA E INGENIERÍA DE DATOS</strong></h2>

<div style="display:flex; justify-content:center; align-items:center; padding:5px;">
        <img src="imgs/upct_logo.png" style="height:300px; width:auto">
    </div>

<h2><strong>Procesamiento del Lenguaje Natural</strong></h2>

<h3><strong>PROYECTO:</strong><br>
    <strong>Aplicación PLN para análisis y procesamiento de texto de carácter geopolítico, económico-bursátil y político.</strong></h3>

<p><strong>ESTUDIANTES</strong></p>
    <ul style="list-style-type:none; padding: 0;">
        <li>Rubén Gil Martínez</li>
        <li>Guillermo López Pérez</li>
    </ul>
</div>


### **1) COMPILACIÓN DEL CORPUS DE COMENTARIOS DE REDDIT A TRAVÉS DE SU API Y USO DE PROCESAMIENTO LÉXICO:**

**LIBRERÍAS**

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import datetime as dt
import time

import praw
import pandas as pd
from datetime import datetime
import time
import re
import os


import nltk



In [2]:
# Configuración de credenciales
reddit = praw.Reddit(
    client_id='v4Ozy7so3ZR5n-i4r7Spqw',
    client_secret='iDAtL0KxYY9ky4GyPWyUM2OAZ4LnBg',
    user_agent='dl-pln-2025-RGM-GLP'
)

In [3]:
# Prueba simple de correcta autenticación
try:
    # Intenta acceder a tu nombre de usuario para verificar autenticación
    print(f"Autenticado como: {reddit.user.me()}")
    
    # Prueba una solicitud simple
    for submission in reddit.subreddit("test").hot(limit=1):
        print(f"Título del post de prueba: {submission.title}")
        
    print("¡Autenticación exitosa!")
    
except Exception as e:
    print(f"Error de autenticación: {type(e).__name__} - {str(e)}")
    print("Por favor, verifica tus credenciales y la configuración de la aplicación.")

Autenticado como: None
Título del post de prueba: Some test commands
¡Autenticación exitosa!


In [ ]:
# Lista de subreddits a extraer
subreddits = ['Economics', 'finance', 'StockMarket', 'geopolitics', 'worldnews', 'business']

# Expresiones regulares para filtrado
url_pattern = re.compile(r'(https?://\S+)')
email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')

# Función para verificar si un comentario es válido
def is_valid_comment(text):
    if len(text) < 30:  # Mínimo de 30 caracteres
        return False
    
    # Eliminar URLs y emails
    text_cleaned = url_pattern.sub(' ', text)
    text_cleaned = email_pattern.sub(' ', text_cleaned)
    
    # Verificar si queda suficiente contenido
    return len(text_cleaned.strip()) >= 30

**EXTRACCIÓN DEL CORPUS DE COMENTARIOS EN ARCHIVOS JSON INDIVIDUALES PARA CADA SUBREDDIT**

- 6 subreddits distintos
- 20 hilos distintos de cada subreddit
- Alrededor de 50 comentarios por cada hilo

Durante la extracción hemos tenido en cuenta el evitar duplicados, tomar comentarios distribuidos en el tiempo (hemos tomado desde comentarios de la última semana hasta comentarios de otros años), tomar comentarios que no sean muy cortos y que además no contengan ni urls ni gmails ya que estos no aportan información relevante. 

Parte de este filtrado de comentarios ha sido posible gracias a la función is_valid_comment() previamente definida.

In [ ]:
# Procesamiento de cada subreddit
for subreddit_name in subreddits:
    print(f"Procesando subreddit: {subreddit_name}...")
    
    subreddit_data = {
        "subreddit_name": subreddit_name,
        "threads": []
    }

    subreddit = reddit.subreddit(subreddit_name)
    threads_collected = 0  # Contador de hilos extraídos
    extracted_thread_ids = set()  # Para evitar duplicados

    # Diferentes filtros de tiempo
    time_filters = ['all', 'week', 'month', 'year']

    for time_filter in time_filters:
        if threads_collected >= 20:
            break
        
        print(f"  Extrayendo de {subreddit_name} con filtro: {time_filter}")

        try:
            threads = list(subreddit.top(time_filter=time_filter, limit=8))  # Procesamos 8 hilos para cada temporalidad para mayor rapidez y evitar bloqueos
            
            for thread in threads:
                if threads_collected >= 20 or thread.id in extracted_thread_ids:  # Evitar duplicados
                    continue  

                print(f"    Procesando hilo: {thread.title}")

                # Información del hilo
                thread_info = {
                    "thread_id": thread.id,
                    "title": thread.title,
                    "author": str(thread.author),
                    "created_utc": datetime.utcfromtimestamp(thread.created_utc).isoformat(),
                    "score": thread.score,
                    "url": thread.url,
                    "permalink": thread.permalink,
                    "time_filter": time_filter,
                    "subreddit_name": subreddit_name,
                    "comments": []
                }


                # Extraer comentarios
                try:
                    thread.comments.replace_more(limit=3)  # Evita cargar demasiados comentarios
                    flat_comments = thread.comments.list()
                except Exception as e:
                    print(f"      Error al cargar comentarios: {e}")
                    continue

                valid_comments_count = 0

                for comment in flat_comments:
                    if valid_comments_count >= 50:
                        break

                    if is_valid_comment(comment.body): # Si el comentario pasa el filtro de validez es almacenado en la lista de comentarios de su respectivo hilo

                        comment_info = {
                            'comment_id': comment.id,
                            'author': str(comment.author),
                            'text': comment.body,
                            'score': comment.score,
                            'created_utc': datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                            'subreddit_name': subreddit_name,
                            'thread_id': thread.id,
                            'is_submitter': comment.is_submitter,
                            'permalink': comment.permalink
                        }

                        thread_info["comments"].append(comment_info)
                        valid_comments_count += 1

                if valid_comments_count >= 25:  # Mínimo de 25 comentarios válidos, sino, el hilo es descartado
                    subreddit_data["threads"].append(thread_info)
                    extracted_thread_ids.add(thread.id)
                    threads_collected += 1
                    print(f"      Añadido con {valid_comments_count} comentarios válidos")
                else:
                    print(f"      Descartado: solo {valid_comments_count} comentarios válidos")

                time.sleep(1.5)  # Evitar exceso de peticiones a Reddit, nos permite cumplir con las normas de uso de la API y evitar bloqueos temporales

        except Exception as e:
            print(f"Error en {subreddit_name} con filtro {time_filter}: {e}")
            continue

    # Guardar en JSON
    os.makedirs('datasets', exist_ok=True)
    output_file = os.path.join('datasets', f"{subreddit_name}_data_2.0.json")

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(subreddit_data, f, ensure_ascii=False, indent=2)

    print(f"Completado: {subreddit_name} - {threads_collected} hilos guardados en {output_file}\n")

print("Extracción completa.")


Procesando subreddit: Economics...
  Extrayendo de Economics con filtro: all
    Procesando hilo: Economist Warns That Elon Musk Is About to Cause a "Deep, Deep Recession"
      Añadido con 50 comentarios válidos
    Procesando hilo: 65% of Americans support monthly $2,000 COVID stimulus payments, new poll shows
      Añadido con 50 comentarios válidos
    Procesando hilo: US tourism to suffer huge '£49 billion drop' under Donald Trump
      Añadido con 50 comentarios válidos
    Procesando hilo: Fed report finds 75% of $800 billion Paycheck Protection Program didn't reach employees
      Añadido con 30 comentarios válidos
    Procesando hilo: Trudeau announces Canada ban on Russian oil imports. “Today, we are announcing a ban on all imports of crude oil from Russia, an industry that has benefited President Putin and his oligarchs greatly,”
      Añadido con 50 comentarios válidos
    Procesando hilo: Trump's first 50 days mark one of the worst starts for the S&P 500 under any presiden

**PROCESAMIENTO LÉXICO Y CREACIÓN DE UN DATAFRAME PARA TRATAR CON MAYOR FACILIDAD LOS COMENTARIOS**

In [37]:
from symspellpy import SymSpell, Verbosity
import spacy

# Cargamos el modelo de spaCy para realizar la Lematización
nlp = spacy.load('en_core_web_sm')

# Configuración de SymSpell para realizar la corrección automática de errores ortográficos
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = "datasets/frequency_dictionary_en_82_765.txt"  
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, separator=" ")


True

In [38]:
def remove_emojis(text):
    # Patrón para detectar emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
        u"\U0001F680-\U0001F6FF"  # símbolos de transporte & mapas
        u"\U0001F1E0-\U0001F1FF"  # banderas (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

**FUNCIÓN DE PROCESAMIENTO LÉXICO**

In [39]:
def preprocess_text(text, autocorrect=True):
    # Eliminar emojis
    text = remove_emojis(text)
    
    # Convertimos a minúsculas
    text = text.lower()
    
    # Procesamiento con spaCy
    doc = nlp(text)
    
    # Tokenización y eliminación de signos de puntuación y números
    tokens = [token.text for token in doc if not token.is_punct and not token.like_num]
    
    # Corrección ortográfica con SymSpellPy
    if autocorrect:
        corrected_tokens = []
        for token in tokens:
            suggestions = sym_spell.lookup(token, Verbosity.CLOSEST, max_edit_distance=2)
            # Si hay sugerencias, tomamos la primera
            # Si no, mantenemos el token original
            if suggestions:
                corrected_tokens.append(suggestions[0].term)
            else:
                corrected_tokens.append(token)
        tokens = corrected_tokens
    
    # Eliminación de stopwords
    tokens = [token for token in tokens if not token in nlp.Defaults.stop_words]
    
    # Lematización con spaCy
    doc = nlp(' '.join(tokens))
    tokens = [token.lemma_ for token in doc]
    
    return ' '.join(tokens)

## Procesamiento Léxico con spaCy y SymSpellPy

Usamos **spaCy** para el procesamiento completo del texto. Aprovechamos sus capacidades integradas para:

- Tokenización  
- Detección de números y puntuación  
- Uso de su propia lista de *stopwords*  
- Lematización más precisa  

Además, realizamos la **corrección ortográfica** con **SymSpellPy**.

### Ventajas de usar spaCy

El proceso es más eficiente porque spaCy está optimizado para procesamiento de texto. La lematización con spaCy suele ser más precisa que con NLTK porque:

- Considera el contexto de la palabra  
- Tiene mejor manejo de formas irregulares  
- Está más actualizado con el lenguaje moderno  
- Es más rápido en el procesamiento  




**REALIZACIÓN DEL PROCESAMIENTO Y PREPARACIÓN DE LOS COMENTARIOS PARA SU POSTERIOR USO EN DIVERSAS TAREAS**

In [40]:
# Cargamos los datos extraídos y procesamos los comentarios
all_comments = []
dataset_path = 'datasets'

for filename in os.listdir(dataset_path):
    if filename.endswith('_data_2.0.json'):
        with open(os.path.join(dataset_path, filename), 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            for thread in data['threads']:
                for comment in thread['comments']:
                    comment_data = {
                        'body': comment['text'],
                        'score': comment['score'],
                        'created_utc': comment['created_utc'],
                        'thread_title': thread['title'],
                        'subreddit': data['subreddit_name'],
                        'processed_text': preprocess_text(comment['text'], autocorrect=True),
                        'processed_text_no_correction': preprocess_text(comment['text'], autocorrect=False)
                    }
                    all_comments.append(comment_data)

**CREACIÓN DE UN DATAFRAME CON TODOS LOS COMENTARIOS EXTRAÍDOS**

In [ ]:
print(len(all_comments))  # Obtención de alrededor de 6000 comentarios válidos

5891


In [44]:
# Crear DataFrame
df_comments = pd.DataFrame(all_comments)

# Mostrar las primeras filas y información del DataFrame
df_comments.head(10)


,body,score,created_utc,thread_title,subreddit,processed_text,processed_text_no_correction
0,Consumer staples. \n\nChurch and Dwight is mis...,94,2022-12-17T04:52:06,The 11 companies that own everything,business,consumer staple church dwight miss clorox clorox,consumer staple \n\n church dwight miss \n\n ...
1,now show how much their boards overlap to real...,146,2022-12-17T07:41:32,The 11 companies that own everything,business,board overlap trigger nausea,board overlap trigger nausea
2,"""everything"" here is limited to consumable pro...",196,2022-12-17T05:11:13,The 11 companies that own everything,business,limited consumable product fraction overall ec...,limited consumable product fraction overall ec...
3,I work for Mars and this graph only shows thei...,9,2022-12-17T19:39:32,The 11 companies that own everything,business,work mar graph show snack relate holding mar o...,work mar graph show snack relate holding mar o...
4,Missing Blackstone and Apollo asset management 😛,22,2022-12-17T07:41:36,The 11 companies that own everything,business,miss blackstone apollo asset management,miss blackstone apollo asset management
5,"Who owns Dr. Pepper you ask? No one, she’s her...",5,2022-12-17T17:47:49,The 11 companies that own everything,business,own pepper ask woman,owns dr pepper ask woman
6,We won't have a chance at having a representat...,3,2022-12-17T13:04:23,The 11 companies that own everything,business,chance have representative government limit ca...,will chance have representative government lim...
7,Now do this for communication and media please.,3,2022-12-17T15:41:06,The 11 companies that own everything,business,communication medium,communication medium
8,"Um excuse me, I don't know much about business...",3,2022-12-17T09:02:32,The 11 companies that own everything,business,excuse know business lipton pepsi unilever,um excuse know business lipton pepsico uniclever
9,"Mondelez is a bit fluffed up with Cadbury, Cad...",3,2022-12-17T09:08:01,The 11 companies that own everything,business,mondale bit fluff sudbury sudbury dairy milk s...,mondelez bit fluff cadbury cadbury dairy milk ...


In [49]:
print(df_comments.iloc[3456, 0])

First rule of dictatorships everywhere. Your actions (not words) are to control your own population first and foremost to keep power. 

The biggest enemy of Hamas, as an authoritarian political organization, isn't Israel. The biggest enemy is losing power over their population.


In [45]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5891 entries, 0 to 5890
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   body                          5891 non-null   object
 1   score                         5891 non-null   int64 
 2   created_utc                   5891 non-null   object
 3   thread_title                  5891 non-null   object
 4   subreddit                     5891 non-null   object
 5   processed_text                5891 non-null   object
 6   processed_text_no_correction  5891 non-null   object
dtypes: int64(1), object(6)
memory usage: 322.3+ KB


In [47]:
df_comments.to_csv('datasets/comments_data.csv', index=False, header=True, encoding='utf-8')